In [2]:
import numpy as np
import pandas as pd

### data load

In [33]:
import numpy as np
import netCDF4 as nc
import glob

# nwp grid, stn(bias corrected) load, here example use obs data instead of bias corrected nwp
# nwp grid
input_dir = "/Users/mankikim/JOB/DATA/"
files = glob.glob(input_dir + "*.nc")
nwp_grd_u = np.array(nc.Dataset(files[-1]).variables['UGRD_10maboveground'])
print(nwp_grd_u.shape)

nwp_lat = np.load("../DAIO/nwp/ldaps_lat_grid")
nwp_lon = np.load("../DAIO/nwp/ldaps_lon_grid")
print(nwp_lat.shape)
print(nwp_lon.shape)

# # check diff
# dif_ll = []
# #for i in range(nwp_lon.shape[0]):
# #    for j in range(nwp_lon.shape[1]-1):
# for i in range(100):
#     for j in range(100):
#         dif_ll.append( np.round(nwp_lon[i,j+1] - nwp_lon[i,j], 5))
# print(set(dif_ll))

# obs stn
obs_stnd = np.load("../DAIO/obs/merge/obs_data")
print(obs_stnd.shape)


# select U & V
#if sel_v == 'u':
#    nwp_grid[:,:,0]

(49, 781, 602)
(781, 602)
(781, 602)
(868, 48, 2)


### make for input format

### Objective analysis(barnes)

In [ ]:


#  1d_var = [[ lat,lon,feature ]]    grid + stn for specified grid(500m*500m)
input_data =

# extract attribute 
lon_lat_data = input_data[:, 0:2]
qff_values = input_data[:, 2]

In [ ]:
# definition of a 12Â° x 12Â° grid starting at 9Â°W / 47Â°N
resolution = 32.0
step = 1.0 / resolution
x0 = np.asarray([-9.0, 47.0], dtype=np.float64)
size = (int(12.0 / step), int(12.0 / step))

# calculate Barnes interpolation
from fastbarnes import interpolation
sigma = 1.0
field = interpolation.barnes(lon_lat_data, qff_values, sigma, x0, step, size)